### Part 4. Langchain - Agent
- Objectives: Langchain 라이브러리를 활용하여 Agent기능 알아보기

In [ ]:
!pip install langchain pinecone-client datasets openai tiktoken

In [ ]:
import os

import time
os.environ['PINECONE_API_KEY']='<YOUR_API_KEY>'
os.environ['OPENAI_API_KEY']='<YOUR_API_KEY>'
pinecone_api_key = os.environ.get('PINECONE_API_KEY')
openai_api_key = os.environ.get('OPENAI_API_KEY')

In [ ]:
#기반 데이터셋 로드
from datasets import load_dataset
dataset = load_dataset("lcw99/wikipedia-korean-20221001", split='train[:100]')

In [ ]:
# 청킹 메서드 정의
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    separators=["\n\n", "\n", " ", ""]
)

In [ ]:
# 임베딩 모델 정의
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=openai_api_key
)

In [ ]:
# 파인콘 인덱스 연결
from pinecone import Pinecone, PodSpec

api_key = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=api_key)

In [ ]:
# 인덱스 만들기

from time import sleep

index_name = 'quickstart'
dimension = 1536
metric = 'dotproduct'
spec = PodSpec('gcp-starter')

if index_name in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

pc.create_index(index_name, dimension=dimension, metric=metric, spec=spec)

while not pc.describe_index(index_name).status['ready']:
    sleep(1)
index = pc.Index(index_name)
sleep(1)
index_stats = index.describe_index_stats()
print(index_stats)

In [ ]:
#업서트 하기
from tqdm.auto import tqdm
from uuid import uuid4

def process_and_upload_records(dataset, batch_limit=100):
    texts = []
    metadatas = []

    for i, record in enumerate(tqdm(dataset)):
        metadata = {
            'id': str(record['id']),
            'source': record['url'],
            'title': record['title']
        }

        record_texts = text_splitter.split_text(record['text'])
        record_metadatas = [
            {"chunk": j, "text": text, **metadata}
            for j, text in enumerate(record_texts)
        ]

        texts.extend(record_texts)
        metadatas.extend(record_metadatas)

        if len(texts) >= batch_limit:
            upload_data(texts, metadatas)
            texts, metadatas = [], []

    if texts:
        upload_data(texts, metadatas)

def upload_data(texts, metadatas):
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeds = embed.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeds, metadatas))

process_and_upload_records(dataset)


In [ ]:

index_name = 'quickstart'
index = pc.Index(index_name)

In [ ]:
#Langchain에서 벡터스토어 연결성 맺기
from langchain.vectorstores import Pinecone

text_field = "text"

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [ ]:
query = "지미 카터가 누구야?"

vectorstore.similarity_search(
    query,
    k=3
)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name='gpt-4-turbo-preview',
    temperature=0.0
)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [ ]:
qa.run(query)

In [ ]:
# 에이전트 없이 질문해보기
qa.run('오늘 야식 메뉴좀 추천해줘')

In [ ]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Pinecone VectorDB',
        func=qa.run,
        description=(
            'use this tool when answering general knowledge queries to get '
            'more information about the topic'
        )
    )
]

In [ ]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [ ]:
# 정상적인 질문해보기
agent(query)

In [ ]:
# 다시 쓸데없는 질문 해보기
agent("오늘 야식 메뉴 추천좀 해줘")